# Data Driven Business project

-
-
- Rick van der Kleij
- Mathijs de Jong (V2B)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from cleaning import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from customAccuracy import *
import joblib

### Importeren dataset

In [ ]:
# Load the data
df = load_data('sap_storing_data_hu_project.csv')

# Data understanding

In [ ]:
df.describe()

# Data cleaning

In [ ]:
# Define the columns to drop
columns_to_drop = ['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld',
                   'stm_scenario_mon', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd',
                   'stm_aanntpl_ddt', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt',
                   'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz',
                   'stm_veroorz_groep', 'stm_veroorz_code', 'stm_veroorz_tekst_kort', 'stm_effect', 'stm_afspr_aanvangddt',
                   'stm_mon_eind_ddt', 'stm_mon_vhdsincident', 'stm_dir_betrok_tr', 'stm_aangelegd_dd', 'stm_aangelegd_tijd',
                   'stm_mon_begindatum', 'stm_mon_begintijd', 'stm_progfh_gw_datum', 'stm_mon_eind_datum', 'stm_mon_eind_tijd',
                   'stm_controle_dd', 'stm_akkoord_mon_toewijz', 'stm_status_sapnaarmon', 'stm_fact_jn', 'stm_akkoord_melding_jn',
                   'stm_afsluit_ddt', 'stm_afsluit_dd', 'stm_afsluit_tijd', 'stm_rec_toegev_ddt', 'stm_hinderwaarde',
                   'stm_actie', 'stm_standplaats', 'stm_status_gebr', 'stm_wbi_nummer', 'stm_projnr', 'stm_historie_toelichting',
                   'stm_schade_verhaalb_jn', 'stm_schadenr', 'stm_schade_status_ga', 'stm_schade_statusdatum', 'stm_relatiervo_vorig',
                   'stm_relatiervo_volgend', 'stm_relatiervo', 'stm_afspr_func_hersteldd', 'stm_afspr_func_hersteltijd',
                   'stm_sorteerveld', 'stm_rapportage_maand', 'stm_rapportage_jaar', 'stm_x_bron_publ_dt', 'stm_x_bron_bestandsnaam',
                   'stm_x_bron_arch_dt', 'stm_x_actueel_ind', 'stm_x_run_id', 'stm_x_bk', 'stm_x_start_sessie_dt', 'stm_x_vervallen_ind']

# Drop the unnecessary columns
df = drop_columns(df, columns_to_drop)

# Clean the data (handle missing values, remove columns with excessive NaNs)
df, avg_list, mode_list = clean_data(df)

# Preprocess the data
df = preprocess_data(df)

# Save the cleaned data to a new CSV (optional)
save_data(df, 'final_db_cleaned.csv')

# Optionally display results
print("Data cleaning and preprocessing complete.")

In [ ]:
# Converteer prognose invoer kolommen naar bruikbare datetime objecten.
#df['stm_progfh_in_invoer_dat'] = pd.to_datetime(df['stm_sap_meld_ddt'].dt.date)
#df['stm_progfh_in_invoer_tijd'] = pd.to_timedelta(df['stm_progfh_in_invoer_tijd'])

# Maak een nieuwe kolom voor het tijdstip waarop de aannemer zijn prognose invult.
#df['prognose_invoer_tijdstip'] = df['stm_progfh_in_invoer_dat'] + df['stm_progfh_in_invoer_tijd']

# Maak een nieuwe kolom aan voor de targetvariabele, de tijd tussen het invullen van de prognose door de aannemer en functieherstel.
#df['target'] = df['stm_fh_ddt'] - df['prognose_invoer_tijdstip']

# Verwijder alle rijen met een negatieve target. In deze gevallen heeft de aannemer zijn prognose na het functieherstel ingevuld, en is de data niet te gebruiken om op te trainen.
#df = df[df['target'] >= pd.Timedelta(0)]

In [ ]:
# Testen
#df[['stm_sap_meld_ddt', 'stm_fh_ddt', 'prognose_invoer_tijdstip', 'totale_functiehersteltijd', 'target', 'stm_aanntpl_tijd']].sample(10)

In [ ]:
#testen 2
df[['stm_fh_duur', 'stm_progfh_in_duur']].sample(10)

In [ ]:
# Maak kolom voor de targetvariabele, de tijd tussen aannemer ter plaatse en functieherstel
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format="%H:%M:%S", errors='coerce')

# Records waar stm_aanntpl_tijd niet is ingevuld verwijderen (stm_aanntpl_tijd == 00:00:00 in deze gevallen)
df = df[df['stm_aanntpl_tijd'] != pd.to_datetime("00:00:00").time()]

# Kolom targetvariabele aanmaken
df['target2'] = df['stm_fh_ddt'] - df['stm_aanntpl_tijd']

# Selecteer alleen de tijd van target2
df['target2'] = df['target2'].dt.components.apply(lambda x: f"{x.hours:02}:{x.minutes:02}:{x.seconds:02}", axis=1)

# Records met target van 0 minuten verwijderen (stm_aanntpl_tijd == stm_fh_ddt in deze gevallen)
df = df[df['target2'] != pd.to_datetime("00:00:00").time()]

# Target converteren naar minuten zodat het in het model gebruikt kan worden
# Convert `target2` to seconds
df['target2'] = (df['target2'] / pd.Timedelta(minutes=1)).astype(int)


df[['stm_sap_meld_ddt', 'stm_fh_ddt', 'totale_functiehersteltijd', 'stm_aanntpl_tijd', 'target2', 'stm_fh_duur']].sample(10)

In [ ]:
# # Inspecteer de unieke waarden in de kolommen
# print("Unieke waarden in stm_geo_mld:")
# print(df['stm_geo_mld'].unique())
# print("Unieke waarden in stm_sap_meldtijd:")
# print(df['stm_sap_meldtijd'].unique())

# # Omzetten van stm_geo_mld naar numeriek
# df['stm_geo_mld'] = pd.to_numeric(df['stm_geo_mld'], errors='coerce')
# nan_count_geo_mld = df['stm_geo_mld'].isnull().sum()
# print(f"Aantal NaN-waarden in stm_geo_mld: {nan_count_geo_mld}")

# # Vul NaN-waarden in stm_geo_mld met de gemiddelde waarde
# df['stm_geo_mld'].fillna(df['stm_geo_mld'].mean(), inplace=True)

# # Tijd omzetten naar seconden voor stm_sap_meldtijd
# def time_to_seconds(t):
#     # Controleer of de waarde niet leeg of NaN is
#     if pd.notnull(t) and t != '':
#         # Probeer de tijdstring te splitsen en om te zetten naar int
#         try:
#             h, m, s = map(int, t.split(':'))
#             return h * 3600 + m * 60 + s
#         except ValueError:
#             return np.nan  # Retourneer NaN bij een fout
#     return np.nan  # Retourneer NaN als de waarde leeg is

# # Pas de functie toe op de kolom stm_sap_meldtijd
# df['stm_sap_meldtijd'] = df['stm_sap_meldtijd'].apply(time_to_seconds)

# # Controleer de types van de kolommen in de dataset
# print("Kolomtypes na cleaning:")
# print(df.dtypes)

# Baseline model

In [ ]:
# Verwijder spaties aan het begin en einde van de waarden
df['stm_progfh_in_duur_clean'] = df['stm_progfh_in_duur'].str.strip()
# Vervang ongeldige waarden door NaN en converteer naar numeriek
df['stm_progfh_in_duur_clean'] = pd.to_numeric(df['stm_progfh_in_duur_clean'], errors='coerce')
# Vul NaN-waarden in met de gemiddelde waarde (zonder inplace=True)
df['stm_progfh_in_duur_clean'] = df['stm_progfh_in_duur_clean'].fillna(df['stm_progfh_in_duur_clean'].mean())

# Calculate the difference between total time and projected time by contractor as the baseline model
df['difference'] = df['stm_fh_duur'] - df['stm_progfh_in_duur_clean']

print(df['difference'].mean())

In [ ]:
# Mean Absolute Error (MAE)
mae = df['difference'].abs().mean()
print(f'Mean Absolute Error (MAE): {mae}')


In [ ]:
# Calculate the squared differences
squared_diff = (df['difference'])**2

# Calculate the RMSE
rmse = np.sqrt(squared_diff.mean())

print(rmse)


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code']]
y = df['stm_fh_duur']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineair regressiemodel aanmaken
model = LinearRegression()

# Train het model
model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred = model.predict(X_test)

# Bereken de evaluatiestatistieken
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}') 
print(f'R² waarde: {r2}') 

# Plot de voorspellingen
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, color='blue', label='Voorspellingen')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', linestyle='--', label='Ideale voorspelling')
plt.title('Voorspellingen vs. Werkelijke Waarden')
plt.xlabel('Werkelijke Waarden')
plt.ylabel('Voorspellingen')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code']]
y = df['target2']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor aanmaken met beperkte diepte
dt_model = DecisionTreeRegressor(max_depth=3, random_state=42)

# Train het model
dt_model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred_dt = dt_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print(f'Mean Squared Error van Decision Tree: {mse_dt}') 
print(f'R² waarde van Decision Tree: {r2_dt}') 

# Visualiseer de Decision Tree
plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=X.columns, filled=True)
plt.title('Decision Tree voor stm_fh_duur (max_depth=3)')
plt.show()


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code', 'stm_geo_mld', 'stm_prioriteit', 'stm_sap_meldtijd']]
y = df['target2']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor aanmaken met beperkte diepte
dt_model = DecisionTreeRegressor(max_depth=3, random_state=42)

# Train het model
dt_model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred_dt = dt_model.predict(X_test)

# Bereken de evaluatiestatistieken
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print(f'Mean Squared Error van Decision Tree: {mse_dt}') 
print(f'R² waarde van Decision Tree: {r2_dt}') 

# Gebruik de functie om de nauwkeurigheid met een specifieke tolerantie te berekenen
tolerance_minutes = 10  # Stel de tolerantie in minuten in
accuracy = custom_accuracy(y_test, y_pred_dt, tolerance_minutes)
print(f'Nauwkeurigheid van Decision Tree binnen ±{tolerance_minutes} minuten: {accuracy:.2f}%')

# Visualiseer de Decision Tree
plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=X.columns, filled=True)
plt.title('Decision Tree voor stm_fh_duur (max_depth=3)')
plt.show()


In [ ]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur_clean', 'stm_oorz_code', 'stm_geo_mld', 'stm_prioriteit', 'stm_sap_meldtijd']]
y = df['target2']

# Splitsen van de dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instellen van de parameters voor Grid Search
param_grid = {
    'max_depth': [3, 5, 10, 20],  # Verschillende diepten van de bomen
    'n_estimators': [100],  # We houden het aantal estimators constant op 100
}

# Grid Search aanmaken
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')

# Train het model met Grid Search
grid_search.fit(X_train, y_train)

# Beste model en parameters
best_rf_model = grid_search.best_estimator_
print(f'Beste hyperparameters: {grid_search.best_params_}')

# Sla het beste model op
joblib.dump(best_rf_model, 'beste_random_forest_model.pkl')

# Maak voorspellingen op de testset met het beste model
y_pred_rf = best_rf_model.predict(X_test)

# Belangrijkste kenmerken van het model plotten
importances = best_rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
features = X.columns

plt.figure(figsize=(10, 6))
plt.title("Feature Importances in Random Forest Model")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), [features[i] for i in indices], rotation=45)
plt.xlabel("Features")
plt.ylabel("Belang")
plt.show()

In [ ]:
# Limiteer de voorspellingen en werkelijke waarden tot maximaal 480 minuten
max_duration = 480
y_pred_binned = np.clip(np.round(y_pred_rf / 10) * 10, 0, max_duration)
y_test_binned = np.clip(np.round(y_test / 10) * 10, 0, max_duration)

# Bereken de nauwkeurigheid binnen deze limiet
accuracy = accuracy_score(y_test_binned, y_pred_binned)
print(f"Nauwkeurigheid binnen 10-minuten-bins (max 8 uur): {accuracy:.2f}")

# Unieke bins en frequenties berekenen tot 480 minuten
bins = np.unique(y_test_binned)
bin_counts = [(y_test_binned == bin).sum() for bin in bins if bin <= max_duration]

# Distributie van voorspellingen en werkelijke waarden plotten tot 480 minuten
plt.figure(figsize=(12, 6))
plt.hist(y_pred_binned, bins=np.arange(0, max_duration + 10, 10), alpha=0.5, label='Voorspellingen')
plt.hist(y_test_binned, bins=np.arange(0, max_duration + 10, 10), alpha=0.5, label='Werkelijke waarden')
plt.xlabel("Duur in minuten (gebundeld in 10-minuten-bins)")
plt.ylabel("Frequentie")
plt.legend()
plt.title("Distributie van voorspellingen en werkelijke waarden in 10-minuten-bins (tot 8 uur)")
plt.show()